In [1]:
# you must populate this dict with the schools required -> try talking to the teaching team about this

schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests

def get_comments_school(school):
  TAG_RE = re.compile(r'<[^>]+>')
  # defines url to make api call to data -> dynamic with school if you want to scrape competition
  url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
  #makes get request and converts answer to json
  data = requests.get(url).json()
  #converts json to dataframe
  reviews =  pd.DataFrame(data['content']['reviews'])
  
  #aux function to apply regex and remove tags
  def remove_tags(x):
    return TAG_RE.sub('',x)
  reviews['review_body'] = reviews['body'].apply(remove_tags)
  reviews['school'] = school
  return reviews


In [2]:
# could you write this as a list comprehension? ;)
comments = []

for school in schools.keys():
    print(school)
    comments.append(get_comments_school(school))

comments = pd.concat(comments)

ironhack
app-academy
springboard


In [3]:
comments

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,createdAt,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,266276,Rebecca Norén,False,None,2020.0,False,UX-designer,Kickstarted my programming skills,"<span class=""truncatable""><p>As I UX Designer ...",7/9/2020,2020-07-09,Web Development Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,As I UX Designer I’m in touch with programming...,ironhack
1,266264,William,False,None,2020.0,False,Front-End Developer,Amazing choice if you're willing to deep dive ...,"<span class=""truncatable""><p>In a nutshell: gr...",7/9/2020,2020-07-09,Web Development Bootcamp,{'image': None},4.7,[],5.0,5.0,4.0,"In a nutshell: great staff, courses in English...",ironhack
2,266263,Camille Libert,False,None,2020.0,False,Insights Project Manager,Great organisation and adaptability of the cla...,"<span class=""truncatable""><p>Pros:<br>- class ...",7/9/2020,2020-07-09,Data Analytics Bootcamp,{'image': None},4.7,[],5.0,5.0,4.0,"Pros:- class organised with exercises, formal ...",ironhack
3,266037,Anonymous,True,None,2020.0,True,,Great experience as expected :),"<span class=""truncatable""><p>Excellent choice ...",7/3/2020,2020-07-03,Data Analytics Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,Excellent choice to get introduced into Data A...,ironhack
4,266009,Manon Philippe,False,None,2020.0,False,Designer,A very complete bootcamp !,"<span class=""truncatable""><p>I was unconvinced...",7/2/2020,2020-07-02,UX/UI Design Bootcamp,{'image': None},4.7,[],5.0,5.0,4.0,I was unconvinced when I joined this formation...,ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,234894,Stephanie S.,False,None,2015.0,True,Website Manager-UX Lead,Excellent and comprehensive UX course,"<span class=""truncatable""><p>The UX Design Wor...",12/15/2016,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,The UX Design Workshop gave me a great underst...,springboard
593,234877,Joe Fang,False,None,2015.0,True,UX Designer,Springboard helped me start my career in UX,"<span class=""truncatable""><p>Just some backgro...",12/15/2016,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,"Just some background information on myself, I ...",springboard
594,234838,Zeina,False,None,2016.0,True,Senior Graphic Designer,Highly recommended,"<span class=""truncatable""><p>I enjoyed the pro...",12/15/2016,2016-12-15,UX Design,{'image': None},3.7,[],4.0,4.0,3.0,"I enjoyed the program very much, gained a lot ...",springboard
595,242681,Jean,False,None,2016.0,True,Lead Designer/Creative Director,A way to stay current for the highly motivated!,"<span class=""truncatable""><p>This was my first...",12/12/2016,2016-12-12,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,This was my first online experience and it did...,springboard


In [4]:
comments.isnull().sum()

id                    0
name                  0
anonymous             0
hostProgramName    2155
graduatingYear        4
isAlumni              0
jobTitle           1429
tagline               1
body                  0
createdAt             0
queryDate             0
program             179
user                  0
overallScore          8
comments              0
overall              10
curriculum           47
jobSupport          241
review_body           0
school                0
dtype: int64

In [5]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id    
    
    # how could you write a similar block of code to the above in order to record the school ID?

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)

ironhack


C:\Users\Diogo\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


app-academy
springboard


In [6]:
### Tabela Location

In [16]:
import numpy as np

In [14]:
def generate_id(x):
    if str(x) != "Anonymous":
        #re_name=re.findall('([a-z])',str(x.lower()))#
        generate_id=x[0:3].lower()+str(np.random.choice(99999, 1, replace=False))[1:-1]
    else:
        generate_id="ano"+str(np.random.choice(99999, 1, replace=True))[1:-1]
    return generate_id

In [106]:
location = pd.concat(locations_list)
location_bridge = location.copy()

In [120]:
location = location.drop(columns = ['id','description','country.abbrev','city.keyword','state.id','state.name','state.abbrev','state.keyword','school','school_id'],  axis =1)
location

,country.id,country.name,city.id,city.name
0,57.0,Germany,31156.0,Berlin
1,29.0,Mexico,31175.0,Mexico City
2,59.0,Netherlands,31168.0,Amsterdam
3,42.0,Brazil,31121.0,Sao Paulo
4,38.0,France,31136.0,Paris
5,1.0,United States,31.0,Miami
6,12.0,Spain,31052.0,Madrid
7,12.0,Spain,31170.0,Barcelona
8,28.0,Portugal,31075.0,Lisbon
9,NaN,NaN,NaN,NaN


In [121]:
#
location['country.name'].fillna('Online', inplace = True)
location['city.name'].fillna('Online', inplace = True)
location['country.id'].fillna(-1, inplace = True)
location['city.id'].fillna(-1, inplace = True)
location

,country.id,country.name,city.id,city.name
0,57.0,Germany,31156.0,Berlin
1,29.0,Mexico,31175.0,Mexico City
2,59.0,Netherlands,31168.0,Amsterdam
3,42.0,Brazil,31121.0,Sao Paulo
4,38.0,France,31136.0,Paris
5,1.0,United States,31.0,Miami
6,12.0,Spain,31052.0,Madrid
7,12.0,Spain,31170.0,Barcelona
8,28.0,Portugal,31075.0,Lisbon
9,-1.0,Online,-1.0,Online


In [122]:
location = pd.DataFrame(location.groupby(['country.id','country.name','city.id','city.name']).first())
location

Empty DataFrame
Columns: []
Index: [(-1.0, Online, -1.0, Online), (1.0, United States, 13.0, San Francisco), (1.0, United States, 31.0, Miami), (1.0, United States, 1507.0, New York City), (12.0, Spain, 31052.0, Madrid), (12.0, Spain, 31170.0, Barcelona), (28.0, Portugal, 31075.0, Lisbon), (29.0, Mexico, 31175.0, Mexico City), (38.0, France, 31136.0, Paris), (42.0, Brazil, 31121.0, Sao Paulo), (57.0, Germany, 31156.0, Berlin), (59.0, Netherlands, 31168.0, Amsterdam)]

In [123]:
location = location.reset_index()
location

,country.id,country.name,city.id,city.name
0,-1.0,Online,-1.0,Online
1,1.0,United States,13.0,San Francisco
2,1.0,United States,31.0,Miami
3,1.0,United States,1507.0,New York City
4,12.0,Spain,31052.0,Madrid
5,12.0,Spain,31170.0,Barcelona
6,28.0,Portugal,31075.0,Lisbon
7,29.0,Mexico,31175.0,Mexico City
8,38.0,France,31136.0,Paris
9,42.0,Brazil,31121.0,Sao Paulo


In [124]:
location["location_id"]=location["city.name"].apply(generate_id)
location

,country.id,country.name,city.id,city.name,location_id
0,-1.0,Online,-1.0,Online,onl36806
1,1.0,United States,13.0,San Francisco,san95474
2,1.0,United States,31.0,Miami,mia71262
3,1.0,United States,1507.0,New York City,new61418
4,12.0,Spain,31052.0,Madrid,mad83753
5,12.0,Spain,31170.0,Barcelona,bar41312
6,28.0,Portugal,31075.0,Lisbon,lis52266
7,29.0,Mexico,31175.0,Mexico City,mex31613
8,38.0,France,31136.0,Paris,par49574
9,42.0,Brazil,31121.0,Sao Paulo,sao22960


In [125]:
#ALterando a posição da coluna location_id
cols =["location_id",'country.id','country.name','city.id','city.name']
location = location[cols]
location

,location_id,country.id,country.name,city.id,city.name
0,onl36806,-1.0,Online,-1.0,Online
1,san95474,1.0,United States,13.0,San Francisco
2,mia71262,1.0,United States,31.0,Miami
3,new61418,1.0,United States,1507.0,New York City
4,mad83753,12.0,Spain,31052.0,Madrid
5,bar41312,12.0,Spain,31170.0,Barcelona
6,lis52266,28.0,Portugal,31075.0,Lisbon
7,mex31613,29.0,Mexico,31175.0,Mexico City
8,par49574,38.0,France,31136.0,Paris
9,sao22960,42.0,Brazil,31121.0,Sao Paulo


In [126]:
#ALterando a posição da coluna location_id
cols =["location_id",'country.id','country.name','city.id','city.name']
location = location[cols]
location

,location_id,country.id,country.name,city.id,city.name
0,onl36806,-1.0,Online,-1.0,Online
1,san95474,1.0,United States,13.0,San Francisco
2,mia71262,1.0,United States,31.0,Miami
3,new61418,1.0,United States,1507.0,New York City
4,mad83753,12.0,Spain,31052.0,Madrid
5,bar41312,12.0,Spain,31170.0,Barcelona
6,lis52266,28.0,Portugal,31075.0,Lisbon
7,mex31613,29.0,Mexico,31175.0,Mexico City
8,par49574,38.0,France,31136.0,Paris
9,sao22960,42.0,Brazil,31121.0,Sao Paulo


In [127]:
#Já tem a coluna school_id 
courses = pd.concat(courses_list)
courses.head(10)

,courses,school,school_id
0,Data Analytics Bootcamp,ironhack,10828
1,Data Analytics Part-Time,ironhack,10828
2,UX/UI Design Bootcamp,ironhack,10828
3,UX/UI Design Part-Time,ironhack,10828
4,Web Development Bootcamp,ironhack,10828
5,Web Development Part-Time,ironhack,10828
0,Bootcamp Prep,app-academy,10525
1,Software Engineer Track: In-Person,app-academy,10525
2,Software Engineer Track: Online,app-academy,10525
0,Data Analytics Career Track,springboard,11035


In [21]:
### Tabela Badge

In [73]:
badge = pd.concat(badges_list)
badge.head(10)
badge_bridge = badge.copy()

In [74]:
badge_bridge

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy,10525
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [78]:
#exclui as colunas 'keyword', 'school'
badge = badge.drop(columns = ['keyword', 'school','school_id' ],  axis =1)

#Altera nomes da colunas 
badge.rename({'name': 'badge_name','description': 'bagde_description'}, inplace = True, axis=1)

#badges["badge_id"]=badges_2["badge_name"].apply(generate_id)
badge

,badge_name,bagde_description
0,Available Online,<p>School offers fully online courses</p>
1,Verified Outcomes,<p>School publishes a third-party verified out...
2,Flexible Classes,<p>School offers part-time and evening classes...
0,Available Online,<p>School offers fully online courses</p>
1,Job Guarantee,<p>School guarantees job placement</p>
0,Available Online,<p>School offers fully online courses</p>
1,Flexible Classes,<p>School offers part-time and evening classes...
2,Job Guarantee,<p>School guarantees job placement</p>


In [79]:
#Removido <p> e </p>
badge['bagde_description'] = badge['bagde_description'].str.lstrip('</p>').str.rstrip('</p>')
badge

,badge_name,bagde_description
0,Available Online,School offers fully online courses
1,Verified Outcomes,School publishes a third-party verified outcom...
2,Flexible Classes,School offers part-time and evening classes
0,Available Online,School offers fully online courses
1,Job Guarantee,School guarantees job placement
0,Available Online,School offers fully online courses
1,Flexible Classes,School offers part-time and evening classes
2,Job Guarantee,School guarantees job placement


In [80]:
#Agrupando badge_name e badge_description
badge = pd.DataFrame(badge.groupby(['badge_name'])['bagde_description'].first())
badge

,bagde_description
badge_name,
Available Online,School offers fully online courses
Flexible Classes,School offers part-time and evening classes
Job Guarantee,School guarantees job placement
Verified Outcomes,School publishes a third-party verified outcom...


In [81]:
badge = badge.reset_index()
badge

,badge_name,bagde_description
0,Available Online,School offers fully online courses
1,Flexible Classes,School offers part-time and evening classes
2,Job Guarantee,School guarantees job placement
3,Verified Outcomes,School publishes a third-party verified outcom...


In [82]:
#Criação do badge_id
badge["badge_id"]=badge["badge_name"].apply(generate_id)
badge

,badge_name,bagde_description,badge_id
0,Available Online,School offers fully online courses,ava52133
1,Flexible Classes,School offers part-time and evening classes,fle52048
2,Job Guarantee,School guarantees job placement,job69803
3,Verified Outcomes,School publishes a third-party verified outcom...,ver61883


In [83]:
cols =["badge_id",'badge_name','bagde_description']
badge = badge[cols]
badge

,badge_id,badge_name,bagde_description
0,ava52133,Available Online,School offers fully online courses
1,fle52048,Flexible Classes,School offers part-time and evening classes
2,job69803,Job Guarantee,School guarantees job placement
3,ver61883,Verified Outcomes,School publishes a third-party verified outcom...


In [33]:
### Tabela School

In [34]:
school = pd.concat(schools_list)
school.head()

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>App Academy is a ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035


In [36]:
#Alterando o nome school por school_name
school.rename({'school': 'school_names'}, inplace = True, axis=1)
school


,website,description,LogoUrl,school_names,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>App Academy is a ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035


In [37]:
#Excluindo colunas website, description e LogoUrl
school = school.drop(columns = ['website','description','LogoUrl'],  axis =1)
school

,school_names,school_id
0,ironhack,10828
0,app-academy,10525
0,springboard,11035


In [38]:
#Alterando a ordem das colunas
cols =["school_id",'school_names']
school = school[cols]
school

,school_id,school_names
0,10828,ironhack
0,10525,app-academy
0,11035,springboard


In [39]:
### Tabela School_badge brigde

In [87]:
badge_bridge

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy,10525
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [85]:
badge

,badge_id,badge_name,bagde_description
0,ava52133,Available Online,School offers fully online courses
1,fle52048,Flexible Classes,School offers part-time and evening classes
2,job69803,Job Guarantee,School guarantees job placement
3,ver61883,Verified Outcomes,School publishes a third-party verified outcom...


In [90]:
school_badge = badge_bridge.merge(badge, how = 'inner', left_on ="name", right_on = 'badge_name')
school_badge

,name,keyword,description,school,school_id,badge_id,badge_name,bagde_description
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828,ava52133,Available Online,School offers fully online courses
1,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525,ava52133,Available Online,School offers fully online courses
2,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035,ava52133,Available Online,School offers fully online courses
3,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828,ver61883,Verified Outcomes,School publishes a third-party verified outcom...
4,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828,fle52048,Flexible Classes,School offers part-time and evening classes
5,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035,fle52048,Flexible Classes,School offers part-time and evening classes
6,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy,10525,job69803,Job Guarantee,School guarantees job placement
7,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035,job69803,Job Guarantee,School guarantees job placement


In [92]:
school_badge = school_badge.drop(columns = ['name', 'keyword', 'description', 'school',
       'badge_name', 'bagde_description'],  axis =1)
school_badge

,school_id,badge_id
0,10828,ava52133
1,10525,ava52133
2,11035,ava52133
3,10828,ver61883
4,10828,fle52048
5,11035,fle52048
6,10525,job69803
7,11035,job69803


In [42]:
### Tabela School_location_bridge

In [107]:
location_bridge

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,NaN,ironhack,10828


In [128]:
location

,location_id,country.id,country.name,city.id,city.name
0,onl36806,-1.0,Online,-1.0,Online
1,san95474,1.0,United States,13.0,San Francisco
2,mia71262,1.0,United States,31.0,Miami
3,new61418,1.0,United States,1507.0,New York City
4,mad83753,12.0,Spain,31052.0,Madrid
5,bar41312,12.0,Spain,31170.0,Barcelona
6,lis52266,28.0,Portugal,31075.0,Lisbon
7,mex31613,29.0,Mexico,31175.0,Mexico City
8,par49574,38.0,France,31136.0,Paris
9,sao22960,42.0,Brazil,31121.0,Sao Paulo


In [135]:
school_location = location.merge(location_bridge, how = 'inner', left_on ="country.name", right_on = 'country.name')
school_location

,location_id,country.id_x,country.name,city.id_x,city.name_x,id,description,country.id_y,country.abbrev,city.id_y,city.name_y,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,san95474,1.0,United States,13.0,San Francisco,16375,"Miami, FL, United States",1.0,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
1,san95474,1.0,United States,13.0,San Francisco,15704,"San Francisco, CA, United States",1.0,US,13.0,San Francisco,san-francisco,6.0,California,CA,california,app-academy,10525
2,san95474,1.0,United States,13.0,San Francisco,15705,"New York City, NY, United States",1.0,US,1507.0,New York City,new-york-city,34.0,New York,NY,new-york,app-academy,10525
3,mia71262,1.0,United States,31.0,Miami,16375,"Miami, FL, United States",1.0,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
4,mia71262,1.0,United States,31.0,Miami,15704,"San Francisco, CA, United States",1.0,US,13.0,San Francisco,san-francisco,6.0,California,CA,california,app-academy,10525
5,mia71262,1.0,United States,31.0,Miami,15705,"New York City, NY, United States",1.0,US,1507.0,New York City,new-york-city,34.0,New York,NY,new-york,app-academy,10525
6,new61418,1.0,United States,1507.0,New York City,16375,"Miami, FL, United States",1.0,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
7,new61418,1.0,United States,1507.0,New York City,15704,"San Francisco, CA, United States",1.0,US,13.0,San Francisco,san-francisco,6.0,California,CA,california,app-academy,10525
8,new61418,1.0,United States,1507.0,New York City,15705,"New York City, NY, United States",1.0,US,1507.0,New York City,new-york-city,34.0,New York,NY,new-york,app-academy,10525
9,mad83753,12.0,Spain,31052.0,Madrid,16376,"Madrid, Spain",12.0,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828


In [136]:
school_location.columns

Index(['location_id', 'country.id_x', 'country.name', 'city.id_x',
       'city.name_x', 'id', 'description', 'country.id_y', 'country.abbrev',
       'city.id_y', 'city.name_y', 'city.keyword', 'state.id', 'state.name',
       'state.abbrev', 'state.keyword', 'school', 'school_id'],
      dtype='object')

In [137]:
school_location = school_location.drop(columns = ['country.id_x', 'country.name', 'city.id_x',
       'city.name_x', 'id', 'description', 'country.id_y', 'country.abbrev',
       'city.id_y', 'city.name_y', 'city.keyword', 'state.id', 'state.name',
       'state.abbrev', 'state.keyword', 'school'],  axis =1)
school_location_bridge

,location_id,school_id
0,san95474,10828
1,san95474,10525
2,san95474,10525
3,mia71262,10828
4,mia71262,10525
5,mia71262,10525
6,new61418,10828
7,new61418,10525
8,new61418,10525
9,mad83753,10828


In [134]:
### tabela School_course_bridge